# Golomb coding for geometrically distributed data

This tutorial will study a variable length entropy coding technique known as *Golomb coding* which constitutes and attractive alternative to Huffman coding in situations where computational resources are constrained, yet a good degree of coding efficiency needs to be retained. Golomb coding was invented to code efficiently data which have a geometric probability mass function. Thanks to the fact that some data associated with image and video coding algorithms are often geometrically distributed, Golomb coding has found its way in different International image and video coding standards.

The tutorial will start by introducing the precursor of Golomb coding, that is the comma code. This will be extended in light of some considerations on the nature of geometric probability mass functions to arrive to the formulation of Golomb codes. From there, we will review some approaches which can be considered to make this coding scheme optimal and/or more adaptive to actual data which may not always follow a geometric distribution. As usual, this tutorial will assume that the reader is familiar with some preliminary concepts related to the [Shannon's information theory](http://people.math.harvard.edu/~ctm/home/text/others/shannon/entropy/entropy.pdf). Good references where these topics can be learned and/or brushed off are as follows:
 * David S. Taubman and Micheal W. Marcellin, "JPEG 2000: Image compression fundamentals, standards and practice", Kluwer Academic Press, 773 pages, 2002.
 * David Solomon, "Data compression, the complete reference, 3rd edition", Springer, xx+899 pages, 2004.
 * Matteo Naccari, "Video coding tutorial (vct): calculation of the entropy for sources with and without memory", [Jupyter notebook](../entropy-calculation.ipynb).

## Comma codes
Consider a memoryless source $S = \{A_S, p_S\}$ where its alphabet $A_S$ is constituted by non negative integers numbers, i.e. $A_S\subseteq \mathbb{N}$ and the probability mass function (**pmf**) $p_S(s)$ is given as:
$$
p_S(s) = 2^{-(s+1)}
$$
For this source of information a Huffman code can be designed where the shortest codeword will be assigned to the symbol with highest probability of apperance, in this case $s\equiv 0$. An alternative code can be constructed as follows:
 * Each codeword associated with symbol $s$, $c_s$, has length equal to $s + 1$ bits
 * The first $s$ bits of $c_s$ are zeros and the last one is a one.

The following table reports a few examples of codewords generated by such a coding scheme:

| Value      | Codeword |
| ----------- | ----------- |
| 0 | 1 |
| 1 | 01 |
| 2 | 001 |
| 3 | 0001 |
| .. | .. |

The coding scheme is denoted as ***comma code*** since the bit "**1**" may be considered as a comma which separates the different codewords. Often the comma code is also referred as [unary code](https://en.wikipedia.org/wiki/Unary_coding) which is widely used to binarise integer quantities undergoing binary arithmetic coding (see the [Jupyter notebook tutorial on arithmetic coding](../arithmetic-coding/practical-schemes/practical-ac-schemes.ipynb) for an example of unary coding employed to this purpose).

Still on the source $S$ considered, we wonder what the rate associated with a comma code is. This is given as the expected length of all codewords:
$$
R = \sum_{s\in A_S}(s + 1)\cdot p_S(s) = \sum_{s\in A_S} \log_2(2^{(s + 1)})\cdot p_S(s) = \sum_{s\in A_S} -\log_2(2^{-(s + 1)})\cdot p_S(s) = -\sum_{s\in A_S}\log_2(p_S(s))\cdot p_S(s) = H(S)
$$

Accordingly, we can see that the comma code achieves optimal (minimum) entropy for the special case of the source $S$ considered. This result should not be too surprising: in fact we note that the comma code can be seen as the output of the Huffman algorithm used over the pmf of $S$. We know from the theory that Huffman codes can achieve Shannon's entropy when the source's pmf is represented with only negative powers of two, hence any Huffman code for source $S$ will also guarantee minimum entropy and indeed the comma code is one instance of the Huffman's algorithm in this particular case. This latter consideration should also be hinting at the reasons behind the claim made in the introduction to this tutorial, i.e. that comma and Golomb codes are usually preferrable when computational resources are constrained. Indeed we may see that a simpler coding scheme which doesn't require any lookup table operation and codebook's design can be used without any coding efficiency penalty.

## Practical example with comma codes
We can numerically verify the theoretical result on the comma (unary) code's average length by simulating a source with the same pmf of $S$. To do this we'll use the Python class for [discrete random variables](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rv_discrete.html) provided by the `scipy` package. The following cell implements the generation of a set of values in the range \[0, 10\) with the given pmf. You will notice a binary flag `reproducible` whose value either determines whether to load the random values from a `numpy` file or generate them on the fly. Such a flag has been added to have the code of this notebook generating the same values listed in the text.

In [ ]:
import numpy as np
from scipy.stats import rv_discrete

rho, n, values = 0.5, 100000, 10
reproducible = True

if reproducible:
    x_geom = np.load("geometric-distribution.npy")
else:
    v = [i for i in range(values)]
    pmf = np.array([rho**(x + 1) for x in v])
    # renormalise
    pmf /= np.sum(pmf)
    rv = rv_discrete(values=(v, pmf))
    x_geom = rv.rvs(size=n)
    np.save("geometric-distribution.npy", x_geom)


We note a *renormalisation* operation when the data are generated (i.e. the `reproducible` flag is `false`). This is required to avoid that the `rv_dicrete` class will throw an exception in case the pmf doesn't add up to 1. It has been observed that this phenomenon seldom happens, however it is still possible. Worth also mentioning that the random data could have also been generated using the `geometric` method from the  [`numpy.random`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.geometric.html) package. Since for the geometric pmf we'll use a convention different from the one assumed in the `numpy` package, we preferred to use the discrete random variable class from `scipy`.

Over the data generated, we first compute the Shannon's entropy to have the minimium amount of bits required to variable length code source $S$. The following Python code cell defines a function to compute the entropy and also returns the histogram associated with the input data.

In [ ]:
# Compute the Shannon's entropy
def compute_entropy(x):
    data_size = x.size
    u = np.unique(x)
    h = []
    for value in u:
        h.append(len(np.where(x == value)[0]) / data_size)
    entropy = -np.sum(np.multiply(h, np.log2(h)))
    return entropy, h

H, h = compute_entropy(x_geom)
print(f"Data entropy: {H:.4f} bps")

The resulting entropy is about about 1.99 bits per symbol. Given that our source emits randomly ten different values, at least four bits are required to represent the data fixed length coding. In case variable length coding is used, a coding efficiency of up to 2:1 can be achieved. We will now verify whether a comma (unary) code can attain such a lower bound as calculated earlier. The following Python code cell defines a function (`unary_codebook`) to compute the comma code for a given set of non negative integer values. The average length is then computed from the data histogram derived above.

In [ ]:
# Build the unary code
def unary_codebook(x):
    u = np.unique(x)
    codebook = {value : "0"*(value) + "1" for value in u}
    return codebook

codebook = unary_codebook(x_geom)
print(codebook)
total_bits = 0

for idx, value in enumerate(codebook.keys()):
    total_bits += len(codebook[value]) * h[idx]

print(f"Total bits for unary coding: {total_bits:.4f}")

As we can see, the average number of bits obtained is very close to the Shannon's entropy and it can get even closer in case we generate a larger set of data. The reader is encouraged to play with variable `n` from the first Python code cell. The code cell above also prints the codebook to appreciate the code generated by the `unary_codebook` function.

## Geometric distributions
The probability mass function of source $S$ can be seen as a special case of [*geometric distrbution*](https://en.wikipedia.org/wiki/Geometric_distribution) with probability $q$ equal to 0.5. A geometric distribution models the probability of success of an event $e$ with probability $q$ to happen after $k+1$ consecutive attempts. Formally we have:

$$
p(k) = (1-q)^{k}\cdot q,
$$

where $p$ is the usual pmf. Differently from the usual definition, we have assumed to perform $k+1$ trials rather than $k$: this would simply change the exponent from $k$ to $k-1$, given that in this latter case we would have $k-1$ failures before being successful. As an example, let the probability of a customer to leave a positive review for a restaurant be $q=0.6$. The owner is interested in the probability of having a positive feedback after four negative reviews. This is given by: $p(4) = (1 - 0.6)^4 \cdot 0.6 = 0.0153$. In another example, consider the probability $q$ to find a corrupted Member of the Italian Parliament during the Berlusconi's government (2001 - 2006). In those years corruption among the MPs was quite high given that Berlusconi got elected all his friends involved in alleged corruption, bribery, etc. Let's then say that $q=0.8$. Considering to pick random members of the Parliament in those years, the probability of finding a corrupted member first at the 6th trial (pun intended) would be: $p(5) = (1 - 0.8)^5 \cdot 0.8 = 2.56\cdot10^{-4}$, that is quite small, because, given the utter immorality brought by Berlusconi in the Italian Government, you didn't need to ask five consecutive MPs before finding one who was alleged in any corruption charge.

We will now rewrite the formulation of a geometric distribution in a manner which will make our following discussion easier. In particular, we set $\rho = 1 - q$, which yields:

$$
p(k) = (1 - \rho)\cdot\rho^k
$$

We see again that by simply setting $\rho = 0.5$ we found again our pmf of the source $S$ considered previously. The reader should make a mental note of this particular value for $\rho$, given that it will be used when introducing the Golomb codes. Before doing so, we would like to check whether our initial unary code scheme is still efficient when the pmf of source $S$ assumes (for example) $\rho = 0.95$. The following Python code cell generates such a geometric distribution.

In [ ]:
# Generate a new geometric distribution with rho very different from 0.5
rho_high = 0.95
v = [i for i in range(values)]
pmf_high = np.array([(1 - rho_high) * rho_high**x for x in v])
# renormalise
pmf_high /= np.sum(pmf_high)
rv_high = rv_discrete(values=(v, pmf_high))
x_geom_high = rv_high.rvs(size=n)


We can now compute the entropy of this different geometric distribution with the following Python code cell.

In [ ]:
# Compute its entropy
entropy_high, h_high = compute_entropy(x_geom_high)
print(f"Entropy geometric distribution with rho = {rho_high} is {entropy_high:.4f} bps")

We note that our source is now more difficult to encode: in fact we move from a 4 bits per symbol if fixed length coding is used to a mere 3.3 in case an optimal coding scheme is employed. We wonder what a unary coding scheme can do in this case, hence:

In [ ]:
# Build the unary code for this case
codebook_high = unary_codebook(x_geom_high)
print(codebook_high)
total_bits_high = 0

for idx, value in enumerate(codebook_high.keys()):
    total_bits_high += len(codebook_high[value]) * h_high[idx]

print(f"Total bits unary coding and geometric distribution with p = {rho_high} are: {total_bits_high:.4f}")

We notice how in this case unary coding would require even more bits than fixed length coding, leading to a significantly worse coding efficiency. This main result motivated Prof. [Solomon Golomb](https://en.wikipedia.org/wiki/Solomon_W._Golomb) to introduce a new coding scheme which it is named after him.

## Golomb codes
Consider again a source of information $X$ with alphabet $A_X \subseteq \mathbb{N}$ and geometric probability mass function $p_X(x) = (1-\rho)\cdot\rho^x$. The starting point of Golomb coding is to write each outcome $x$ using its quotient ($x_q$) and reminder ($x_r$) upon the division by $m$, that is:

$$
x = m\cdot x_q + x_r
$$

Given that $X$ is a random variable, also its quotient and reminder outcomes will define two random variables denoted as: $X_q$ and $X_r$. The former follows a geometric distribution:

$$
p_{X_q}(x_q) = \sum_{i=0}^{m-1}p_X(m\cdot x_q + i) = \sum_{i=0}^{m-1}(1 - \rho)\cdot\rho^{m\cdot x_q + i} = (1 - \rho)\cdot\rho^{m\cdot x_q}\sum_{i=0}^{m-1}\rho^i,
$$

which is parametrised in $\rho^{m}$ whilst $X_r$ is a uniformly distributed random variable. With the reference to the following figure and intuitively we can prove the statements we made for $X_q$ and $X_r$ by noting that $X_q$ will be nothing but the index of a $m$-wide bin resulting from the division upon $m$. The value of such an index (i.e. $x_q$) still follows a geometric pmf. Subsequentely we also notice that within each bin the amount of pmf enclosed can be seen as nearly uniform, most notably if the value for $m$ is small.

<img src="geom-xq-xr.png" alt="Geometric distribution of Xq and Xr" width="1000"/>

It's now time to recall the mental note we made when reviewing comma codes. We said that for geometrically distributed data with $\rho = 0.5$, unary coding achieves the Shannon's entropy. Our $X_q$ has $\rho^m$ as parameter, hence we would have:

$$
\rho^m = 0.5.
$$

In other words we should choose $m$ such that $\rho^m \approx 0.5$. Being $X_r$ uniform, fixed length coding would suffice. Accordingly, for a given number $x$ generated by our source $X$, and value $m$ chosen, the Golomb coding procedure can be summarised as follows:
 * Compute $x_q$, $x_r$ and $k = \lceil\log_2(m)\rceil$
 * Write down the unary representation of $x_q$
 * If $x_r < 2^k - m$ then encode $x_r$ using its binary representation with $k-1$ bits else encode $x_r$ using its binary representation with $k$ bits
 * Concatenate the string of bits associated with $x_q$ and $x_r$ and output the result as Golomb code for $x$

The case where $m$ is a power of two and $m = 2^k$ is special because it will require exactly $k$ bits for $x_r$. Moreover, in such a case the calculation of $x_q$ merely resembles a binary shift whilst $x_r$ is given by the bits discarded during the binary shift. Golomb codes where $m$ is a power of 2 are often denoted as *Golomb-Rice* codes and $k$ is said to be the *Golomb-Rice* parameter.

## Golomb coding: Practical example
The following Python code cell defines a method to compute the Golomb-Rice codebook over a given source's alphabet $A_S$. We will then use such a method to compute the codebook for a given value of $m$ and then apply it to the outcomes of our geometric source where $\rho=0.95$. A dedicated Python code cell will output the average number of bits required for different values of $k$.

In [ ]:
def golomb_codebook(x, k):
    u = np.unique(x)
    m = 1 << k
    codebook = {}
    for value in u:
        q = value // m
        r = value % m
        prefix = "0"*q + "1"
        suffix = bin(r)[2:].zfill(k) if k else ""
        codebook[value] = prefix + suffix if value else prefix
    return codebook

In [ ]:

for k in range(8):
    gc_codebook = golomb_codebook(x_geom_high, k)
    total_bits_high = 0

    for idx, value in enumerate(gc_codebook.keys()):
        total_bits_high += len(gc_codebook[value]) * h_high[idx]
    print(f"Total bits for Golomb-Rice coding with k = {k}: {total_bits_high:.4f} bits")

## Selection of the optimal Golomb-Rice parameter
For eight different values of $k$ we obtained a variable number of average bits. We note the minimum number of bits attained for $k=2$ and so we wonder whether there is way to determine $k$ beforehand so to guarantee that our variable length coding uses the minimum number of bits.

We start by considering the expected value for a geometric source which is given by:

$$
E[X] = \sum_{x=0}^\infty(1 - \rho)\cdot x\rho^x = (1 - \rho)\cdot\sum_{x=1}^\infty(x-1)\cdot\rho^{x-1} = \left((1 - \rho)\frac{d}{d\rho}\sum_{x=0}^\infty\rho^x\right) - (1 - \rho)\sum_{x=0}^\infty\rho^x = \left((1 - \rho)\frac{d}{d\rho}\sum_{x=0}^\infty\rho^x\right) - 1 = \frac{\rho}{1 - \rho}.
$$

Assuming that $(1 - \rho) \ll 1$ and remembering the Taylor-Maclaurin expansion for $(1 + x)^\alpha$ we have:

$$
\rho^m = (1 - (1 - \rho))^m \approx 1 - m\cdot(1 - \rho).
$$

From the result obtained for the expected value of $X$, we have:

$$
\rho^m \approx 1 - \frac{m\cdot\rho}{E[X]} \approx 1 - \frac{m}{E[X]},
$$

we saw that $\rho^m \approx 0.5$, thus:

$$
\rho^m \approx \frac{1}{2} \Rightarrow 1 - \frac{m}{E[X]} \approx \frac{1}{2} \Rightarrow m \approx \frac{1}{2}\cdot E[X].
$$

Recalling that $m = 2^k$ we would have $k = \log_2(0.5\cdot E[X])$, or more precisely:

$$
k = \max\left\{0, \left\lceil\log_2\left(\frac{1}{2}\cdot E[X]\right)\right\rceil\right\}.
$$

The following Python code cell computes the optimal $k$ according to the result above. Over our geometric distribution with $\rho=0.5$ we indeed obtain $k=2$.

In [ ]:
mu = np.mean(x_geom_high)
k_opt = max(0, np.ceil(np.log2(0.5 * mu)))
print(f"Mean is: {mu:.2f} Optimal k is: {int(k_opt)}")

## Adaptive Golomb-Rice coding
The result obtained above for the optimal value of $k$ assumes that the statistics of the information source $X$ do not vary, i.e. the process $X$ is stationary. Image and video data are definitely non stationary, hence an adaptive coding scheme would compensate for the variability of $E[X]$. The following Python pseudo-code shows an archetype of adaptive Golomb-Rice coding which served as the basis for practical encoding of the prediction residuals in the JPEG-LS coding standard.

```Python
A = mean_estimate_for_X, N = 1
for x in X:
    k = max(0, ceil(log2(0.5 * A / N)))
    codeword = golomb_rice(x, k)
    write(codeword)
    if N == Nmax:
        A //= 2
        N //= 2
    N += 1
    A += x
```
The value `Nmax` denotes a windowing function to make the algorithm more or less reactive to the statistics' variability. The reader may have noticed that again this adaptive algorithm relies on an estimate for $E[X]$. As we will see in the practical example later, such estimate does not need to be really accurate, given that the algorithm will reach easily convergence, especially on a large number of data. For the sake of completeness, we shall mention that the same adaptive mechanism is also performed at the decoder's side in lockstep so to have the transmitter and receiver in sync.

## Practical example: run length encoding
We said at the beginning of this tutorial that often some data associated with image and video content are geometrically distributed. One type of these data can be obtained by considering all consecutive occurences of a given pixel value $v$. Each of these continuous repetitions can be replaced by a pair $(r, v)$ where $r$ identifies the length of a *run* of value $v$. This simple idea is the starting point of the very well-known entropy coding technique denoted as *Run Length Encoding* ([RLE](https://en.wikipedia.org/wiki/Run-length_encoding)). In its simplest form, RLE may be using a fixed number of bits to represent $r$ and $v$. Without loss of generality, let's assume that this number of bits it's the same for both $r$ and $v$ and is equal to the same number of bits used to represent each pixel, say $b$. If the average length of a run of consecutive pixels is $\mu_r$, the compression ratio associated with RLE over an image with $T$ pixels is then as follows:

$$
\large
CR = \frac{T\cdot b}{\frac{T}{\mu_r}\cdot b \cdot 2} = \frac{\mu_r}{2}.
$$

One can do a better job by using variable length coding applied to $r$ and $v$. Let's focus on $r$: we notice that if a given value $v$ repeats quite often in long runs, its probability to appear is $q \approx 1$ and there will be long runs of $v$ interspersed by a value different from $v$. Restricted to the case whether the value is $v$ or not we see that the runs associated with $v$ follow a geometric distribution with parameter $\rho = 1 - q$, for which we know that Golomb coding is well suited. A little thought should convince the reader that the consideration made for value $v$ can be extended to all values composing our image: consider an image representing content grabbed from a computer screen with plenty of pixels repeated if the image is read according to a raster scan order. Worthwhile recalling that RLE is the only way to achieve a fractional number of coding bits when using Huffman coding. We will now measure the coding efficiency of run length encoding when Golomb codes are used to encode the different runs computed over a typical desktop content, which is more likely to contain large runs compared to a natural content where the camera noise may easily interrupt the runs.

### Preliminaries and benchmarks
The following Python code cell loads the test image we will be using in our tutorial: this is the Wikipedia page in Italian for the Rex liner, the only Italian liner winning the Blue Riband and the largest Italian transatlantic ever build until 1931.

The image is loaded in memory and then subsampled by a factor of 2 with a simple nearest neighbour kernel to speed up the computations. The red, green and blue planes are then combined together to obtain the luma component by applying the simple Reversible Colour Transform ([RCT](https://en.wikipedia.org/wiki/JPEG_2000#Color_components_transformation)) as specified in the JPEG 2000 standard.

In [ ]:
import cv2
import matplotlib.pyplot as plt
image = cv2.imread('../../input-data/rex-wikipedia.png', cv2.IMREAD_UNCHANGED).astype(np.int32)
stride = 2
if len(image.shape) == 2:
    # Take just the luma component
    image = image[::stride, ::stride]
else:
    # Apply RCT to get the luma component
    r, g, b = image[::stride, ::stride, 2], image[::stride, ::stride, 1], image[::stride, ::stride, 0]
    image = (r + 2 * g + b) >> 2

plt.figure(figsize=(20, 15))
plt.imshow(image, cmap="gray")
plt.title("Subsampled input image");

The following Python code cell defines a function to collect the run length and value pairs over a given image plane. The convention used here is that a single value $v$ run will be represented with the pair $(0, v)$. We note that such a collection could have also been achived by using the `groupby` function from the `itertools` package.

In [ ]:
def run_length_collect(image):
    width = image.shape[1]
    total_elements, idx = image.size, 0
    run, length = [], []

    while idx < total_elements:
        r, c = idx // width, idx % width
        crun = image[r, c]
        clength = 0
        stop_search = False
        while not stop_search and idx < total_elements:
            idx += 1
            r, c = idx // width, idx % width
            if idx != total_elements and crun != image[r, c]:
                run.append(crun)
                length.append(clength)
                stop_search = True
            elif idx == total_elements:
                run.append(crun)
                length.append(clength)
            clength += 1

    return run, length

def compute_frequency_table(data):
    ft = []
    for e in np.unique(data):
        idx = np.where(data == e)
        frequency = len(idx[0])
        ft.append((e, frequency))
    return ft

We are now ready to collect the run length and value pairs and we will study the coding efficiency of the run length term. As a benchmark to compare the results against, we will consider Huffman coding applied over the different run lengths. To this end, we will use the `huffman` Python package to compute the codebook. The values of each run will instead be encoded with a fixed number of bits (8): this is because the distribution of image pixels it is unlikely to follow a geometric distribution, hence applying Golomb Coding will produce suboptimal results. We note that this consideration will change in case we apply spatial (intra) prediction to image pixels: in this case the residuals will align more towards a geometric distribution. As a prove of this statement, the JPEG Lossless (JPEG-LS) standard uses an adaptive Golomb coding method to encode the residuals resulting from intra prediction. The following Python code cell computes the number of bits required by Huffman coding when applied over the runs.

In [ ]:
import huffman

value, length = run_length_collect(image)
value = np.array(value, dtype=np.int32)
length = np.array(length, dtype=np.int32)

# Do Huffman encoding of the run lengths
l_ft = compute_frequency_table(length)

l_code = huffman.codebook(l_ft)

bits_total, bits_v, bits_l = 0, 0, 0
for r, l in zip(value, length):
    v_cw = bin(r)[2:].zfill(8)
    l_cw = l_code[l]
    bits_v += len(v_cw)
    bits_l += len(l_cw)
    bits_total += len(v_cw) + len(l_cw)

cr_huffman = image.size * 8 / bits_total
share_v = bits_v / bits_total * 100
share_l = bits_l / bits_total * 100
print(f"Total bits used for Huffman coding: {bits_total}, values used: {share_v:.2f} whilst run lengths used: {share_l:.2f}, compression ratio: {cr_huffman:.2f}")

### Coding efficiency in case of constrained resources
The compression ratio obtained for RLE with Huffman codes constitutes our upper bound in terms of coding efficiency, that is the best coding gain we can achieve. This is because we have assumed that all image pixel values could be loaded in memory so to derive the Huffman's codebook. However, when the computational resources (e.g. memory) are scarce and/or the end-to-end latency needs to be minimised, we may not be able to afford to load the whole image into memory and this statement becomes even more compelling in case of very large resolution images such as 8k resolution and/or hyperspectral data with more than three colour planes.

From what we discussed above, we note that the adaptive Golomb coding method can be an attractive alternative to Huffman codes. In fact, we could load just one image line at the time and process the run length pairs accordingly. In case a run of a given value spans more than a line of pixels, we could imagine to keep counting its length and load the next line. The statistics of the adaptive Golomb method will then be updated when we finish to code one run. The following Python code cell implements the adaptive Golomb coding method presented earlier. We initialise the accumulator variable `A`, with a plausible value which may be inferred by oberserving the first line of image pixels: given that they are associated with the window's title bar, they share the same value, hence it is reasonable to set the length of the run equal to the image's width (i.e. 960). Such a value will then be updated when we process the runs and will reflect the fact that as we move down the image will contain more areas with text, hence the average length of the runs will decrease. We also set our update window period (i.e. `Nmax`) equal to 20% of the total pixels.

In [ ]:
def golomb_coding(x, k):
    if x == 0:
        return "1"
    m = 1 << k
    q, r = x // m, x % m
    prefix = "0" * q
    suffix = bin(r)[2:].zfill(k) if k else ""
    return prefix + "1" + suffix

# Adaptive Golomb coding
n_max = int(image.size * 0.2)
l_accumulator = 960
l_counter = 1

bits_gc, bits_gc_v, bits_gc_l = 0, 0, 0
for r, l in zip(value, length):
    l_log_arg = l_accumulator / (2 * l_counter) if l_accumulator else 0.5
    l_k = int(max(0, np.ceil(np.log2(l_log_arg))))
    v_cw = bin(r)[2:].zfill(8)
    l_cw = golomb_coding(l, l_k)
    bits_gc_v += len(v_cw)
    bits_gc_l += len(l_cw)
    bits_gc += len(v_cw) + len(l_cw)

    # Update counters
    if l_counter == n_max:
        l_accumulator //= 2
        l_counter //= 2

    l_accumulator += l
    l_counter += 1
cr_gc = image.size * 8 / bits_gc
share_r_gc = bits_gc_v / bits_gc * 100
share_l_gc = bits_gc_l / bits_gc * 100
print(f"Total bits used for adaptive Golomb: {bits_gc}, values used: {share_r_gc:.2f} whilst run lengths used: {share_l_gc:.2f}, compression ratio: {cr_gc:.2f}")
print(f"Average accumulator value: {l_accumulator / l_counter:.2f}, actual average run length: {np.mean(length):.2f}")

We see from the compression ratio obtained above that the adaptive Golomb method deviates by only 7% from Huffman coding's efficiency with the great advantage that this approach doesn't require all image data to be loaded into memory and then to compute large Huffman tables to be transmitted them to the receiver.

We also appreciate how the final value of the average run length adapted by the algorithm very closely approaches the actual one.

### Adapting the statistics within the run
We have seen that adaptive Golomb coding can perform fairly close to Huffman coding without the need to have all image data available and design the codebook tables. We described a potential way of working of such an adaptive method whereby we load into memory a line of image pixels, compute the run length pairs and then apply the method. We can also update the statistics within the run so our RLE system can even be simplified in terms of memory requirements by reading just one sample at the time rather than a full image line. In fact, in such a case we can imagine to update the value of parameter $k$ after emitting each bit of the comma code. Ideally we would reach a situation where the typical (i.e. the average) run with length $E[X]$ will be coded with a single bit and, in case the run length is greater than $E[X]$, we will add the comma bit followed by the remainder value in $k$ bits. The parameter $k$ is increased every time we're still in the run (i.e. its length is increasing) and decreased otherwise. This simple principle constitutes the core of the [MELCODE](https://www.hpl.hp.com/loco/HPL-98-193R1.pdf) method specified in the JPEG-LS standard. The following pseudo-Python-code reports the MELCODE method for a run with value $r$. We note that the pseudo-code may be easily modified to run on a per sample basis rather than receiving the full length of the run $r$ in one go.

```Python
k = melcode_state_machine[state]
m = 1 << k
while r >= m:
    write(0)
    r -= m
    state = min(state_max, state + 1)
    k = melcode_state_machine[state]
    m = 1 << k

write(1)
write(r, k) # Write r using k bits
state = max(0, state - 1)
```

The `melcode_state_machine` is a 32 entry array which is known both at the encoder's and decoder's side and regulates the growth of $k$. The variable `state` is initialised to zero. Worth mentioning that a simplified version of the MELCODE method, denoted as MEL, which uses only thirtheen entries of the `melcode_state_machine` is specified by the [High Throughput JPEG 2000 standard](https://htj2k.com/). A method very similar to the MELCODE algorithm has also been proposed by [Malvar](https://www.researchgate.net/publication/4230021_Adaptive_run-lengthGolomb-Rice_encoding_of_quantized_generalized_Gaussian_sources_with_unknown_statistics) in 2006, which has then been adopted in the [Microsoft RemoteFX codec](https://docs.microsoft.com/en-us/openspecs/windows_protocols/ms-rdprfx/62495a4a-a495-46ea-b459-5cde04c44549).

The actual implementation of the MELCODE method is defined in the following Python code cell. The method implementing MELCODE does not write any bit as specified in the pseudo-code but rather counts the bits an actual implementation would write.

In [ ]:
melcode_state_machine = [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 9, 10, 11, 12, 13, 14, 15]

def mel_encode(run, state, bc):
    k = melcode_state_machine[state]
    m = 1 << k
    run_passed = False
    while run >= m:
        bc[0] += 1  # Output a '0', hit
        run -= m
        state = min(31, state + 1)
        k = melcode_state_machine[state]
        m = 1 << k
        run_passed = True

    bc[0] += 1  # Output a '1', miss
    if k:
        bc[0] += k  # Output the k-bit binary code for r

    if not run_passed:
        state = max(0, state - 1)
    return state

The following Python code cell runs the MELCODE method over the run length pairs associated with our test image. As usual the compression ratio and number of bits used by the value and the run lengths are reported.

In [ ]:
bits_mel, bits_melcode_l, bits_melcode_v = 0, 0, 0
state = 0
for r, l in zip(value, length):
    bc = [0]
    state = mel_encode(l, state, bc)
    v_cw = bin(r)[2:].zfill(8)
    bits_melcode_v += len(v_cw)
    bits_melcode_l += bc[0]
    bits_mel += len(v_cw) + bc[0]

cr_melcode = image.size * 8 / bits_mel
share_mel_r = bits_melcode_v / bits_mel * 100
share_mel_l = bits_melcode_l / bits_mel * 100
print(f"Total bits used for MEL: {bits_mel}, values used: {share_mel_r:.2f} whilst run lengths used: {share_mel_l:.2f}, compression ratio: {cr_melcode:.2f}")

We notice how the *within the run* adaptation made by the MELCODE method can provide a benefit to the whole coding efficiency by buying another 1.8% increment compared to the adaptive Golomb method.

### Zero order exponential Golomb codes
We said that the MELCODE algorithm adapts the Golomb-Rice parameter $k$ to shorten the prefix of the code whenever the run length increases. We recall that the value of the prefix in a Golomb code is the result of the division upon $m = 2^k$. From the figure shown above, we also remember that such a division equals to the quantisation of the probability mass function (**pmf**) of our run length distribution, hence increasing $k$ would correspond to a coarser quantisation of such pmf, with the size of the quantisation bins related to the hard coded rule in the MELCODE's state machine. Teuhola in 1978 [1] proposed to have quantisation bins whose extent grows exponentially. This idea gave the name to the *Exponential Golomb Coding* (EGC) method which, for a run length of value $x$, can summarised as follows:
 1. Find $n$ such that: $\sum_{i=k}^n2^i \leq x < \sum_{i=k}^{n+1}$ with $n \geq k - 1$
 1. Form the prefix of 0-bits with $n-k+1$ bits
 1. Insert the comma 1-bit
 1. Represent $x - \sum_{i=k}^n2^i$ in binary using $n+k$ bits

The following figure shows an example of EGC assuming $k=0$.

<img src="egc-example.png" alt="Exponential Golomb coding example for k = 0" width="1000"/>

We will now apply the EGC method with $k=0$ to the desktop image considered so far. The following Python code cell defines the EGC procedure for $k=0$.

[1] J. Teuhola, "A compression method for clustered bit-vectors", *IEEE Information processing letters*, vol. 7, no 6, pp. 308-311, October 1978.

In [ ]:
def egc0(x):
    if x == 0:
        return "1"
    x += 1
    k = int(np.log2(x))
    r = x % (1 << k)
    prefix = "0" * k
    suffix = bin(r)[2:].zfill(k)
    return prefix + "1" + suffix

The following Python coding cell will instead apply EGC over the run length values collected over the test image. As usual, the compression ratio is measure to be then compared with the ones obtained so far.

In [ ]:
bits_egc, bits_egc_l, bits_egc_v = 0, 0, 0
for v, l in zip(value, length):
    v_cw = bin(v)[2:].zfill(8)
    bits_egc_v += len(v_cw)
    l_cw = egc0(l)
    bits_egc_l += len(l_cw)
    bits_egc += len(v_cw) + len(l_cw)

cr_egc = image.size * 8 / bits_egc
share_egc_l = bits_egc_l / bits_egc * 100
share_egc_v = bits_egc_v / bits_egc * 100
print(f"Total bits used for Exponential Golomb: {bits_egc}, values used: {share_egc_v:.2f} whilst run lengths used: {share_egc_l:.2f}, compression ratio: {cr_egc:.2f}")

We may see that with a method simpler than MELCODE\*, we can obtain a compression ratio which is quite close to Huffman's. Indeed Teuhola points out that his method has superior performance in the case of clustered vectors of zeros (or any other value composing the run) whereas it is not so efficient in the case of purely random generated data. To verify numerically such a statement, the following Python code cell runs EGC and normal Golomb coding over the random geometric distribution generated earlier with $\rho=0.95$. Golomb coding uses $k=2$, given that this was found to be the optimal for that particular distribution.

(\*) MELCODE would imply to store a 32 entry array and perform more operations than EGC as highlighted in the `while` loop from the pseudo-code above. One might argue that a `log2` operation is involved in EGC which may be computational demanding. However since we're just interest in the `floor` part of its result, such a calculation may be easily accomplished on modern CPUs by using *first set bit* type of operations implemented via a *count leading zeros* instruction.

In [ ]:
u = np.unique(x_geom_high)
gc_codebook = golomb_codebook(x_geom_high, 2)
average_rate_egc0, average_rate_gc = 0, 0
for idx, v in enumerate(u):
    cw_eg0 = egc0(v)
    average_rate_egc0 += len(cw_eg0) * h_high[idx]
    average_rate_gc += len(gc_codebook[v]) * h_high[idx]

print(f"Coding bits for EGC(k=0): {average_rate_egc0:.2f}, coding bits for Golomb coding: {average_rate_gc:.2f}")

We may now see that when run length values are randomly generated, EGC underperforms Golomb coding. Thanks to the fact that run lengths in images and videos resemble more closely clustered vectors (as we've just obtained over our test image), EGC managed to find its way in several image and video coding standards such as H.264/AVC, H.265/HEVC and SMPTE VC-2, to mention a few. In particular, for the case of the H.265/HEVC standard the parameter $k$ is also adapted to better fit the current data distribution of the `coeff_abs_remaining_level` syntax levels (see this very good [research paper](https://ieeexplore.ieee.org/document/6324418) for more details).

## Conclusions
Hopefully the reader has enjoyed this tutorial whose takeaways may be summarised as follows:
 * Data which are geometrically distributed maybe compressed efficiently using Golomb coding which resembles (optimal) comma codes in case the geometric distribution has parameter $\rho$ equal 0.5
 * If the statistics of the data source are known beforehand, the optimal value for the Golomb-Rice parameter $k$ may found via a closed form relationship
 * Conversely, to compensate the fact that real data are not stationary, adaptive versions of Golomb coding can be derived

Furthermore, we have also learned that over real image data, considering continuous chunks of pixels which share the same value (i.e. run length and value pairs) Golomb coding, particularly its adaptive variants can achieve a coding efficiency very close to the one associated with Huffman coding with the big advantage of not requiring to compute the codebook table, hence reading the whole image data into memory. We considered run length coding as an example since this is the practical coding scheme used in many image and video compression formats and provides a good case where Huffman coding may be used to achieve fractional coding rates. Among the adaptive variants of Golomb coding we have also introduced one which adapts $k$ within the run. Some additional takeaways on the adaptivity of Golomb coding are as follows:
 * Adaptive Golomb coding works well but it may be slow in adapting to the source's statistics, especially in case the initial parameters are far from the optimal ones and we start by encoding long run lengths
 * The MELCODE method is more efficient but it requires some additional operations on each run length, plus additional memory to store the array associated with the state machine

Finally, we introduced a simpler, yet very effective method to adapt $k$ within the run: EGC. Over real image data this has given the compression ratio closest to Huffman's. We observed that EGC may be inefficient in case the data are randomly distributed. Such a fact should be considered to guide the design of the actual coding engine.